# Three Core Components of PyTorch

<img src="../asssets/a1-three-components-of-pytorch.png"/>

1. **Tensor Library:** Extends the concept of array-oriented programming library, *`NumPy`* with the *`GPU`* support.

2. **Automatic Differentiation Engine `(Autograd)`:** Enables Automatic Calculation of Gradients(Slopes) for Tensor Operations simplifying the process of `backpropagation` and `model optimization`.

3. **Deep Learning Library:** Offers `Modular`, `Flexible` and `Efficient` Building Blocks including `Pretrained Models`, `Loss Functions` and `Optimizers` for designing and training a wide range of deep learning models.


> In the news, **LLMs** are often referred to as **AI models**. However, LLMs are also a type of  **deep  neural  network**,  and **PyTorch**  is  a  deep  learning  library.

<img src="../asssets/ai-ml-dl.png"/>

<img src="../asssets/a3-supervised-learning.png"/>

<img src="../asssets/apple silicon.png"/>

# Tensors

In [37]:
import torch

torch.__version__

'2.2.0'

In [38]:
torch.cuda.is_available()

False

## Understanding Tensors

<img src="../asssets/tensors.png">

## Scalars, Vectors, Matrices and Tensors

<img src="../asssets/create-tensors.png"/>

In [39]:
import torch

tensor0d: torch.Tensor = torch.tensor(data=1)
print(tensor0d)

tensor(1)


In [40]:
import torch

tensor1d: torch.Tensor = torch.tensor(data=[1, 2, 3])
print(tensor1d)

tensor([1, 2, 3])


In [41]:
import torch

tensor2d: torch.Tensor = torch.tensor(data=[[1, 2], [3, 4]])
print(tensor2d)

tensor([[1, 2],
        [3, 4]])


In [42]:
import torch

tensor3d: torch.Tensor = torch.tensor(
    data=[[[1, 2], [3, 4]], [[5, 6], [7, 8]]],
)
print(tensor3d)

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


## Tensor Datatypes

- PyTorch  adopts  the  default  `64-bit  integer`  data  type  from  Python.  We  can  access  the data type of a tensor via the *`.dtype`* attribute of a tensor:

In [43]:
import torch

tensor1d: torch.Tensor = torch.tensor(data=[1, 2, 3])
print(tensor1d)
print(tensor1d.dtype)

tensor([1, 2, 3])
torch.int64


- If we create tensors from Python floats, PyTorch creates tensors with a *`32-bit precision`* by default:

In [44]:
import torch

floatvector: torch.Tensor = torch.tensor(data=[1.0, 2.0, 3.0])
print(floatvector)
print(floatvector.dtype)

tensor([1., 2., 3.])
torch.float32


**This choice is primarily due to the *balance between precision and computational efficiency*. A 32-bit floating-point number offers sufficient precision for most deep learning tasks while consuming less memory and computational resources than a 64-bit floating-point number. Moreover, *GPU architectures are optimized for 32-bit* computations, and using this data type can significantly speed up model training and inference.**

> Moreover, it is possible to change the precision using a tensor’s **`.to`** method.

In [45]:
import torch

floatvector64: torch.Tensor = torch.tensor(data=[1, 2, 3])
print(floatvector64)
print(floatvector64.dtype, "\n")

floatvector32: torch.Tensor = torch.tensor(data=[1, 2, 3]).to(dtype=torch.float32)
print(floatvector32)
print(floatvector32.dtype)


tensor([1, 2, 3])
torch.int64 

tensor([1., 2., 3.])
torch.float32


## Common PyTorch Tensor Operations

- The *`.shape`* attribute allows us to access the shape of a tensor:

In [46]:
import torch

tensor2d: torch.Tensor = torch.tensor(data=[[1, 2, 3], [4, 5, 6]])
print(tensor2d)
print(tensor2d.shape)

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])


As you can see, *`.shape`* returns `[2, 3]`, meaning the tensor has *2 rows* and *3 columns*. To reshape the tensor into a `3 × 2` tensor, we can use the *`.reshape`* method:

In [47]:
print(tensor2d.reshape(3, 2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


However, note that the *more common command for reshaping* tensors in PyTorch is *`.view()`*:

In [48]:
print(tensor2d.view(3, 2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


The key difference between `.view()` and `.reshape()` in PyTorch lies in how they handle memory layout: `.view()` requires the tensor to be **contiguous** (data stored in a continuous block of memory) and will raise an error if it isn’t, as it only *provides a new "view" into the existing data* **without copying it**. In contrast, `.reshape()` works regardless of whether the tensor is contiguous; if needed, it creates a new, contiguous copy of the data to ensure the desired shape. Use `.view()` for efficiency when the tensor is contiguous and `.reshape()` for flexibility.


- We can use **`.T`** to transpose a tensor, which means flipping it across its diagonal. Note that this is similar to reshaping a tensor, as you can see based on the following result:

In [49]:
print(tensor2d)
print(tensor2d.T)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])


The common way to multiply two matrices in PyTorch is the **`.matmul`** method:

In [50]:
print(tensor2d)
print(tensor2d.T)
print(tensor2d.matmul(other=tensor2d.T))

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[14, 32],
        [32, 77]])


We can also adopt the **`@`** operator, which accomplishes the same thing more compactly:

In [51]:
print(tensor2d)
print(tensor2d.T)
print(tensor2d @ tensor2d.T)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[14, 32],
        [32, 77]])


# Autograd

## Seeing Models as Computational Graphs
Now let’s look at PyTorch’s *`automatic differentiation engine`*, also known as *`autograd`*. PyTorch’s autograd system provides *functions to compute gradients (slopes)* in dynamic computational graphs automatically. 

- A **`computational graph`** is a `directed graph` that allows us to **express** and **visualize mathematical expressions**. In the context of deep learning, a computation graph lays out the sequence of calculations needed to compute the output of a neural network we  will  need  this  to  compute  the  required  gradients  for backpropagation,  the  main training algorithm for neural networks.

The code in the following listing implements the **forward pass (prediction step)** of a **simple logistic regression classifier**, which can be seen as a `single-layer neural network`. It returns a score between 0 and 1, which is compared to the true class label (0 or 1) when computing the loss.


<img src="../asssets/logistic-regression-forward-pass.png"/>


In [52]:
import torch
import torch.nn.functional as F

y: torch.Tensor = torch.tensor(data=[1.0])  # True Label

x1: torch.Tensor = torch.tensor(data=[1.1])  # Indepndent Variable
w1: torch.Tensor = torch.tensor(data=[2.2])  # Weight

b: torch.Tensor = torch.tensor(data=[0.0])  # Bias

z: torch.Tensor = x1 * w1 + b  # Linear Function
a: torch.Tensor = torch.sigmoid(input=z)  # Activation Function

loss: torch.Tensor = F.binary_cross_entropy(input=a, target=y)  # Loss

<img src="../asssets/computational-graph.png">

PyTorch builds such a computation graph in the background, and we can use this to *`calculate gradients(slope) of a loss function with respect to the model parameters`* (here **`w1`** and **`b`**) *`to train the model.`*

## Automatic Differentiation Made Easy
If we carry out computations in PyTorch, it will build a computational graph internally by default if one of its terminal  nodes has the **`requires_grad`** attribute  set to `True`. This is useful if we want to compute gradients. **Gradients are required when training neural networks** via the popular **`backpropagation algorithm`**, which can be considered an *`implementation of the chain rule`* from calculus for neural networks.

<img src="../asssets/partial-derivative.png">

## PARTIAL DERIVATIVES AND GRADIENTS

- **`Partial Derivatives:`** measure *`the rate at which a function changes with respect to one of its variables`*. 

- A **`gradient (slope)`** is a *`vector containing all of the partial derivatives of a multivariate function`*, *a function with more than one variable as input*.


> On a high level, the **`Chain Rule`** is a way to *`compute the gradients (slope) of a Loss Function`* given the Model's Parameters in a Computational Graph. This provides the information needed to **update** each of the Model's Parameter to **Minimize the Loss Functi**, which serves as a Proxy for measuring the **Performance** of the Model using **Gradient Descent**.

**So, Why Autograd?**

- It **automatically** builds a `computational graph` for us. How? By tracking every operation performed on Tensors.
- It **automatically** computes the `gradients (slopes)` for us. How? By calling the **`grad`** function, we can compute the `gradients of a loss function` with respect to the model parameters (*`weights and biases`*).

<img src="../asssets/compute-gradients-with-autograd.png"/>


In [53]:
from typing import Tuple
from torch import Tensor
import torch.nn.functional as F
from torch.autograd import grad

y: Tensor = torch.tensor(data=[1.0])  # True Label

x1: Tensor = torch.tensor(data=[1.1])  # Indepndent Variable
w1: Tensor = torch.tensor(data=[2.2], requires_grad=True)  # Weight

b: Tensor = torch.tensor(data=[0.0], requires_grad=True)  # Bias

z: Tensor = x1 * w1 + b  # Linear Function
a: Tensor = torch.sigmoid(input=z)  # Activation Function

loss: Tensor = F.binary_cross_entropy(input=a, target=y)  # Loss

gradients_of_loss_wrt_w1: Tuple[Tensor, ...] = grad(
    outputs=loss,
    inputs=w1,
    retain_graph=True,
)
gradients_of_loss_wrt_b: Tuple[Tensor, ...] = grad(
    outputs=loss,
    inputs=b,
    retain_graph=True,
)

print(gradients_of_loss_wrt_w1)
print(gradients_of_loss_wrt_b)


# More efficient and compact way to compute gradients of the Loss Function with respect to the Model's Parameters
loss.backward()  # Calculates the Gradients of the Loss Function wrt all those Tensors that have requires_grad=True
print(w1.grad, b.grad)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor([-0.0898]) tensor([-0.0817])


*While calling `loss.backward()` how does pytorch knows to calculate the `gradients of` **`loss`** wrt whom?*

When you call `loss.backward()`, it calculates the gradients of the loss with respect to all those tensors that have `requires_grad=True`.

> **Note:** While the Calculus Jargon is a means to explain PyTorch's *`autograd`* component , all we need to take away is the PyTorch takes care of the Calculus for us via the **`.backward()`** method.

# Deep Learning Library

## Implementing Multilayer Neural Networks

> Now we focus on PyTorch as a library for implementing Deep Neural Networks.

<img src="../asssets/multilayer-perceptron.png"/>




> Each Layer can have Multiple Nodes.

When implementing a Neural Network in PyTorch, we can `subclass` the **`torch.nn.Module`** `class` to define our own Custom Network Architecture. This *`Module`* Base Class provides a lot of functionality, making it easier to Build and Train Models. For example, it allows us to *`Encapsualte Layers`* and *`Operations`* and *`Keep Track of the Model's Parameters`*.

Within this Sub-Class, we *`define the Network Layers`* in the *`__init__ constructor`* and specify *`how the Layers interact in the Forward Method`*. The *`Forward Method`* `describes how the Input Data passes through the Network` and `comes together as a Computation Graph.` And the *`Backward Method`* `computes the Gradients of the Loss Function with respect to the Model's Parameters (weights & biases) during Training.`

```python
import torch

class Model(torch.nn.Module):
    pass
```

**Inherited Methods:** 
The class inherits numerous methods from the `Module` base class, including:
- `forward()`: Defines the computation performed at every call.

- `parameters()`: Returns an iterator over module parameters.
- `state_dict()`: Returns a dictionary containing the module's state.
- `load_state_dict()`: Copies parameters and buffers from a state dict.
- `to()`: Moves and/or casts the parameters and buffers.
- `cuda()`, `cpu()`: Moves all model parameters and buffers to the GPU/CPU.
- `train()`, `eval()`: Sets the module in training/evaluation mode.
- Many other utility methods for registering hooks, buffers, and parameters.

<img src="../asssets/multilayer-perceptron-with-2-hidden-layers.png" />

In [54]:
import torch


class NeuralNetwork(torch.nn.Module):
    def __init__(self, number_of_inputs: int, number_of_outputs: int) -> None:
        super().__init__()
        self.layers = torch.nn.Sequential(
            # 1st Hidden Layer
            torch.nn.Linear(in_features=number_of_inputs, out_features=30),
            torch.nn.ReLU(),
            # 2nd Hidden Layer
            torch.nn.Linear(in_features=30, out_features=20),
            torch.nn.ReLU(),
            # Output Layer
            torch.nn.Linear(in_features=20, out_features=number_of_outputs),
        )

    # Shape of the Input Tensor 'x': [Batch Size, Number of Inputs]
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logits = self.layers(x)
        return logits


model = NeuralNetwork(number_of_inputs=50, number_of_outputs=3)

print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


> **Note:** that we used the *`Sequential Class`* when defining our NeuralNetwork Class because it makes our life easier if we have a series of Layers we want to execute in a specific order, as we are doing here. This way, after instantiating *`self.layers = torch.nn.Sequential()`*, in the *`__init__`* constructor, we just have to now call the *`self.layers`* attribute instead of calling each layer individually in the *`NeuralNetwork's forward method`*.

To check the *`Total Number of Trainable Parameters`* in our Model, we can use the `parameters()` Method.

In [55]:
count = 0
for param in model.parameters():
    print(type(param), param.numel())
    count += param.numel()

print(count)

<class 'torch.nn.parameter.Parameter'> 1500
<class 'torch.nn.parameter.Parameter'> 30
<class 'torch.nn.parameter.Parameter'> 600
<class 'torch.nn.parameter.Parameter'> 20
<class 'torch.nn.parameter.Parameter'> 60
<class 'torch.nn.parameter.Parameter'> 3
2213


In [56]:
total_number_of_parameters: int = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print(total_number_of_parameters)

2213


> - **Note:** Each parameter for which `requires_grad=True` counts as one `trainable parameter`.
> - For each parameter *`p`*, the method *`.numel()`* returns the total number of elements in the tensor.

In the case of our Neural Network Model with the preceding two hidden layers, these *`trainable parameters`* can be found in the *`torch.nn.Linear`* layers. **`A Linear layer multiplies the inputs with a weight matrix and adds a bias vector.`** This is sometimes referred to as a **`feedforward`** or **`fully connected layer`**. Based on the `print(model)` call we executed here, we can see that the `first Linear layer  is  at  index  position  0` .

In [57]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [58]:
print(model.layers)

Sequential(
  (0): Linear(in_features=50, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=3, bias=True)
)


In [59]:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)

Parameter containing:
tensor([[ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        [-0.0920, -0.0480,  0.0105,  ..., -0.0923,  0.1201,  0.0330],
        ...,
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509],
        [-0.1250,  0.0513,  0.0366,  ..., -0.1370,  0.1074, -0.0704]],
       requires_grad=True)
torch.Size([30, 50])


The weight matrix here is `[30 x 50]` matrix, and we can see that `requires_grad` is set to `True`, which means its entries are *`trainable`*, this is the default setting for weights and biases in *`torch.nn.Linear`*.

In [60]:
print(model.layers[0].bias)
print(model.layers[0].bias.shape)

Parameter containing:
tensor([-1.3122e-01, -1.0667e-02,  1.0314e-01, -1.8104e-02,  3.1523e-02,
        -1.3763e-01,  1.1643e-01,  9.1198e-02,  9.8382e-02,  7.3479e-02,
        -7.2337e-02, -1.1853e-01,  5.3997e-04,  7.5849e-02, -6.1513e-02,
         3.5053e-02, -1.1154e-02, -1.3147e-02,  3.6492e-02,  1.0322e-01,
         2.9582e-02,  1.0176e-02,  2.2896e-02,  2.6020e-02, -4.5835e-02,
        -1.6127e-02,  3.4467e-02, -1.1141e-01,  9.3445e-05, -1.3079e-01],
       requires_grad=True)
torch.Size([30])


**FORWARD PASS:**

In [61]:
# NOT OPTIMIZED
import torch
from torch import Tensor

torch.manual_seed(seed=123)

X: Tensor = torch.rand(size=(1, 50))
output: Tensor = model(X)  # automatically executes the FORWARD PASS
print(output)

tensor([[-0.0879,  0.1729,  0.1534]], grad_fn=<AddmmBackward0>)


The *`FORWARD PASS`* refers to calculating the output tensors of a neural network from the input tensors. Works by passing the input data through all the layers, starting from the input layer, through hidden layers, and finally to the output layer.


*`These three numbers returned here correspond to a score assigned to each of the three output nodes.`* Notice that the output tensor also includes a **`grad_fn`** value.

**`grad_fn=<AddmmBackward0>`** represents the last used function to compute variable in the computational graph. It means that the tensor we are inspecting was created via Matrix Multipication and Addition Operation. PyTorch will use this information to compute the Gradients of the Loss Function with respect to the Model's Parameters suring **`BackPropagation`**. 

If we just want to use a Network without training or backpropagation, for example, if we use it for prediction after training the model, then constructing this computational graph for backpropagation can be wasteful as it performs unnecessary computations and consumes additional memory. So, when we use model for Prediction(Inference) then the best practice is to us the `torch.no_grad` Context Manager. This tells PyTorch that it doesn't need to keep track of the gradients, which can result in significant savings in memory and computation:

In [62]:
with torch.no_grad():
    output: Tensor = model(X)  # automatically executes the FORWARD PASS
print(output)

tensor([[-0.0879,  0.1729,  0.1534]])


> - **`Logits = Output of the Last Layer`**

In PyTorch, it's common to code models such that they return the outputs of the *`Last Layer (Logits)`* without passing them through *`Non-Linear Activation Function`*. That's because PyTorch's commonly used Loss Functions combine the Softmax (Sigmoid for Binary Classification) operation with the negative log-likelihood loss in a single class. The reason for this is `Numerical Efficiency` and `Stability`. So, if we want to compute Class-Membership Probabilities for our Predictions, we have to call the Softmax Function explicitly:

In [63]:
X.shape

torch.Size([1, 50])

In [64]:
with torch.no_grad():
    out: Tensor = torch.softmax(input=model(X), dim=1)
print(out)

tensor([[0.2801, 0.3635, 0.3565]])


The values can now be interpreted as Class-Membership Probabilities that sum up to 1. The values are roughly equal for this random input, which is expected for a randomly initialized model without training.

## Setting up efficient data loaders

<img src="../asssets/dataloaders.png"/>

We will implement a custom *`Dataset Class`*, which we will use to create a training and a test dataset that we'll then use to create a *training* and *test dataset* that we'll then use to create the dataloaders.

- Creating a simple toy dataset of 5 `training set` examples with two features each. 
- Also creating a tensor containing corresponding class labels: 3 examples belong to class 0 and 2 examples belong to class 1. 
- Also creating `testing set` consisting of 2 entries.

In [65]:
X_train: Tensor = torch.tensor(
    data=[
        [-1.2, 3.1],
        [-0.9, 2.9],
        [-0.5, 2.6],
        [2.3, -1.1],
        [2.7, -1.5],
    ]
)

y_train: Tensor = torch.tensor(
    data=[
        0,
        0,
        0,
        1,
        1,
    ]
)

In [80]:
X_test: Tensor = torch.tensor(
    data=[
        [-0.8, 2.8],
        [2.6, -1.6],
    ]
)

y_test: Tensor = torch.tensor(
    data=[
        0,
        1,
    ]
)

> **`NOTE:`** **PyTorch requires that class labels start with label 0, and the largest class label value should not exceed the number of output nodes minus 1 because Python indexing starts from Zero.**

- For, example if we have Class Labels `0,1,2,3` and `4`, the Neural Network Output Layer must consist of 5 Nodes only.

In [81]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X: Tensor, y: Tensor) -> None:
        super().__init__()
        self.features: Tensor = X
        self.labels: Tensor = y

    def __getitem__(self, index) -> tuple[Tensor, Tensor]:
        one_X: Tensor = self.features[index]
        one_y: Tensor = self.labels[index]
        return one_X, one_y

    def __len__(self) -> int:
        return self.labels.shape[0]


train_ds = ToyDataset(X=X_train, y=y_train)
test_ds = ToyDataset(X=X_test, y=y_test)

print(train_ds.features)
print(train_ds.labels)

tensor([[-1.2000,  3.1000],
        [-0.9000,  2.9000],
        [-0.5000,  2.6000],
        [ 2.3000, -1.1000],
        [ 2.7000, -1.5000]])
tensor([0, 0, 0, 1, 1])


The purpose of the `ToyDataset Class` is to instantiate a `PyTorch DataLoader`. The 3 main components of a **Custom `Dataset` Class** are the:
- **`__init__`** constructor 
- **`__getitem__` method** here, we defined to retrieve exactly one data from features and labels.
- **`__len__` method** here, we retrieve the length of the dataset.

These could be file paths, file objects, database connectors and so on. Since, we created a tensor dataset that sits in memory, we simply assign *`X`* and *`y`* to these attributes, which are placeholders for our tensor objects.



In [82]:
print(len(train_ds))
print(len(test_ds))

5
2


<img src="../asssets/instantiating data loaders.png"/>

In [88]:
from torch.utils.data import DataLoader

torch.manual_seed(seed=123)

train_dataloader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
)


test_dataloader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
)


for idx, (x, y) in enumerate(iterable=train_dataloader):
    print(f"Batch {idx+1}: {x} | {y}")

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) | tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) | tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) | tensor([1])


We specified the Batch Size of 2 here, but the 3rd batch only contains a single example. That's because we have 5 training examples, and 5 is not evenly divisible by 2. 

> **`Note`** that in practice, having a substantially smaller batch as the last batch in a Training Epoch can disturb the Convergence during Training . To prevent this, we set **`drop_last=True`**, which will drop the last batch in Each Epoch.

In [91]:
train_dataloader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True,
)


Now, iterating over the train dataloader, we can see that the last batch is omitted. Previously it had 3 batch now it has 2.

In [92]:
for idx, (x, y) in enumerate(iterable=train_dataloader):
    print(f"Batch {idx+1}: {x} | {y}")


Batch 1: tensor([[-0.9000,  2.9000],
        [ 2.3000, -1.1000]]) | tensor([0, 1])
Batch 2: tensor([[ 2.7000, -1.5000],
        [-0.5000,  2.6000]]) | tensor([1, 0])
